# Sound Magician

## Giới thiệu 
Nâng cao! Có thể bạn đã từng xem một bộ phim hoặc chương trình truyền hình mà các điệp viên siêu hạng sử dụng một số kỹ thuật khó tin để nâng cao hình ảnh pixelated hoặc khôi phục một số dữ liệu bị mất. Tại đây sẽ thực hiện điều tương tự — lần này mục tiêu của bạn là sử dụng hồi quy tuyến tính để khôi phục hoặc 'điền vào' phần bị xóa hoàn toàn của một tệp âm thanh!

Bạn sẽ sử dụng FSDD (Free-Spoken-Digits-Dataset), một tập dữ liệu âm thanh do Zohar Jackson tổng hợp lại khi anh ấy nhận thấy không có nhiều âm thanh được làm sạch (không có khoảng trống, độ dài tương đương, cùng bitrate, cùng tỷ lệ mẫu trên giây, ...) thư viện âm thanh sẵn sàng cho machine learning.

In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_random_state
import scipy.io.wavfile as wavfile

`Tín hiệu Analog là tín hiệu liên tục, đồ thị biểu diễn tín hiệu analog là một đường liên tục (ví dụ sin, cos hoặc đường cong lên xuống bất kỳ). Analog có nghĩa là tương tự, tức là tín hiệu sẽ tương tự về bản chất, nhưng sẽ khác nhau về cường độ tín hiệu lúc sau so với lúc trước.`

`Trong toán học, phép ngoại suy là quá trình ước tính giá trị của một biến trên cơ sở mối quan hệ của nó với biến khác, vượt ra ngoài phạm vi quan sát ban đầu.`

`Đề muốn mình khôi phục lại những đoạn analog bị thiếu bằng ngoại suy`

### Về Âm thanh

Mẫu là các quan sát. Mỗi tệp âm thanh sẽ là một mẫu duy nhất trong tập dữ liệu.

Tìm thêm thông tin về [Mẫu âm thanh tại đây] (https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Mỗi tệp .wav thực sự chỉ là một loạt các mẫu số, được lấy mẫu từ tín hiệu analog. Sampling (Lấy mẫu) là một kiểu rời rạc. Khi chúng ta đề cập đến các 'sample (mẫu)', tức là các quan sát. Khi đề cập đến 'audio sample', chúng ta muốn nói đến "feature (thuộc tính)" thực sự của tệp âm thanh.

Mục tiêu của lab này là sử dụng hồi quy tuyến tính, đa mục tiêu để tạo ra bằng phép ngoại suy phần bị thiếu của tệp âm thanh thử nghiệm.

Mỗi một thuộc tính audio_sample sẽ là đầu ra của một phương trình, là một hàm của phần được cung cấp của audio_samples:

    missing_samples = f(provided_samples)

Bạn có thể thử với lượng âm thanh bạn muốn cắt và để máy tính tạo ra bằng tham số Provided_Portion.

Hãy sử dụng cái này. Đây là lượng tệp âm thanh sẽ được cung cấp, tính bằng phần trăm. Phần trăm còn lại của tệp sẽ được tạo thông qua phép ngoại suy tuyến tính.

In [2]:
Provided_Portion = 0.25

có nghĩa là 75% là X và 25% Y

## 1. Chuẩn bị dữ liệu

Bắt đầu bằng cách tạo một list Python thông thường là `zero`:

Lặp qua tập dữ liệu và tải lên tất cả 50 tệp `0_jackson*.wav` bằng phương thức `wavfile.read()`: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html từ thư mục [data](https://drive.google.com/drive/folders/1UDAvuRfXzcOcpI84neQAmAeffIrJKKBi?usp=sharing).

Hãy cẩn thận! `.read()` trả về một tuple và bạn chỉ quan tâm đến dữ liệu âm thanh chứ không phải sample_rate tại thời điểm này. Bên trong vòng lặp for, chỉ cần nối dữ liệu âm thanh đã tải vào list Python `zero`:

In [3]:
import glob
zero = []
directory = 'data-20221026T160641Z-001/data'
for fname in glob.glob(r'data-20221026T160641Z-001\data\0_jackson_*.wav'):
    sample_rate, data = wavfile.read(fname)
    zero.append(data)
print(zero)

[array([-369, -431, -475, ...,  301,  324,  304], dtype=int16), array([-311,  -91, -140, ...,  378,  357,  333], dtype=int16), array([-314, -303, -332, ..., -355, -343, -322], dtype=int16), array([347, 351, 462, ..., 365, 338, 302], dtype=int16), array([-336,  160,   65, ..., -315, -343, -319], dtype=int16), array([ 354,  442,  610, ..., -312, -336, -333], dtype=int16), array([ 397,  531,  638, ..., -357, -386, -353], dtype=int16), array([ 382,  459,  530, ..., -254, -301, -309], dtype=int16), array([-393,   54,  -71, ...,  319,  340,  313], dtype=int16), array([-311, -363, -318, ..., -239, -305, -304], dtype=int16), array([-316, -336, -342, ..., -442, -424, -310], dtype=int16), array([ 335,  392,  481, ..., -302, -314, -304], dtype=int16), array([-361, -226, -238, ..., -286, -311, -343], dtype=int16), array([-309, -323, -333, ..., -301, -300, -304], dtype=int16), array([ 305,  305,  294, ..., -379, -342, -300], dtype=int16), array([ 342,  452,  546, ..., -356, -353, -332], dtype=int16

In [4]:
# import os
# import scipy.io.wavfile as wavfile

# zero = []
# directory = "data-20221026T160641Z-001/data"
# for fname in os.listdir(directory):
#     if fname.startswith("0_jackson"):
#         fullname = os.path.join(directory, fname)
#         sample_rate, data = wavfile.read(fullname)
#         zero.append( data )
# print(zero)

In [5]:
# import os
# from scipy.io.wavfile import read
# zero = []
# for i in range(50):
#     f = r'C:\Users\Quan Thi Thanh Hoa\AppData\Funix\data-20221026T160641Z-001' + "\data\\0_jackson_"+ str(i) +".wav"
#     file = read(f, 'r')
#     zero.append(file[1])
#     print(zero)

Hãy dành một chút thời gian để chuyển 0 thành một khung dữ liệu (DataFrame). Khi bạn làm như vậy, hãy đặt `dtype` thành `np.int16`, vì các tệp âm thanh đầu vào là 16 bit cho mỗi mẫu. Nếu bạn không biết cách thực hiện việc này, hãy đọc tài liệu tại đây: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Tiếc là những đoạn âm thanh này không được chuẩn hóa độ dài nên chúng ta sẽ phải chia nhỏ chúng để tất cả có cùng độ dài. Vì Pandas sẽ chèn các NAN tại bất kỳ vị trí nào để tạo thành mảng [n_observed_samples, n_audio_samples] hình chữ nhật hoàn hảo nên hãy thực hiện một `dropna` trên trục Y ở đây. Sau đó, chuyển đổi 1 trở lại thành NDArray bằng `yourarrayname.values`:

In [6]:
zero = pd.DataFrame(zero)
zero = zero.dropna(1)
zero.astype(np.int16)
zero = zero.values
zero

C:\Users\Quan Thi Thanh Hoa\AppData\Local\Temp\ipykernel_28072\3353062974.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  zero = zero.dropna(1)


array([[-369, -431, -475, ..., -108, -402, -884],
       [-311,  -91, -140, ..., -422, -462, -460],
       [-314, -303, -332, ..., 3418, 2640, 1202],
       ...,
       [-417,  152,  168, ...,  638,  463,  332],
       [ 330,  382,  389, ...,  602,  708,  701],
       [-312, -335, -338, ...,  644,  815,  580]], dtype=int16)

In [7]:
# zero = pd.DataFrame(zero)
# zero = zero.fillna(0)
# # list_zeros = list_zeros.dropna()
# # list_zero = list_zeros.astype(np.int16)
# col=np.array(zero, np.int16)
# zero = zero.values
# zero

Điều quan trọng là phải biết  (bao nhiêu mẫu audio_samples) độ dài dữ liệu hiện có.

`zero` hiện có shape giống như` [n_samples, n_audio_samples] `, vì vậy hãy lấy số lượng` n_audio_samples` và lưu trữ nó trong biến `n_audio_samples`:

mỗi samples chính là audio_samples có 6273 feature

In [8]:
zero

array([[-369, -431, -475, ..., -108, -402, -884],
       [-311,  -91, -140, ..., -422, -462, -460],
       [-314, -303, -332, ..., 3418, 2640, 1202],
       ...,
       [-417,  152,  168, ...,  638,  463,  332],
       [ 330,  382,  389, ...,  602,  708,  701],
       [-312, -335, -338, ...,  644,  815,  580]], dtype=int16)

In [9]:
print(zero.shape)

(50, 4087)


## 2. Huấn luyện mô hình

Tạo mô hình hồi quy tuyến tính của bạn tại đây và lưu trữ nó trong biến `model`. Đừng vội huấn luyện hoặc làm bất cứ điều gì khác với nó:

In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [11]:
msk = np.random.rand(len(zero)) < 0.8
train = zero[msk]
test = zero[~msk]

In [12]:
from sklearn import linear_model
regr = linear_model.LinearRegression()

Có 50 bản ghi mỗi clip. Bạn chỉ muốn lấy ngẫu nhiên một trong số chúng ra và cái đó sẽ KHÔNG được sử dụng trong quá trình huấn luyện mô hình. Nói cách khác, tệp mà chúng ta sẽ kiểm tra/chấm điểm sẽ là một mẫu không nhìn thấy, độc lập với phần còn lại của training set:

Nếu seed là Không, trả về singleton RandomState được sử dụng bởi np.random. Nếu seed là một int, trả về một cá thể RandomState mới được tạo seed. Nếu seed đã là một phiên bản RandomState, hãy trả lại nó. Nếu không, tăng ValueError.

In [13]:
rng = check_random_state(7)

random_idx = rng.randint(zero.shape[0])
test  = zero[random_idx]
train = np.delete(zero, [random_idx], axis=0)

In ra shape của `train` và `test`.

`train` sẽ có dạng:`[n_samples, n_audio_samples]`, trong đó `n_audio_samples` là 'features' (thuộc tính) của tệp âm thanh

`test` sẽ có dạng `[n_audio_features]`, vì nó là mẫu duy nhất (tệp âm thanh, ví dụ: quan sát).

In [14]:
test

array([-417,  152,  168, ...,  638,  463,  332], dtype=int16)

In [15]:
test.shape

(4087,)

In [16]:
train

array([[-369, -431, -475, ..., -108, -402, -884],
       [-311,  -91, -140, ..., -422, -462, -460],
       [-314, -303, -332, ..., 3418, 2640, 1202],
       ...,
       [ 364,  420,  469, ..., -697, -714, -898],
       [ 330,  382,  389, ...,  602,  708,  701],
       [-312, -335, -338, ...,  644,  815,  580]], dtype=int16)

In [17]:
train.shape

(49, 4087)

Dữ liệu thử nghiệm sẽ có hai phần, `X_test` và` y_test`.

`X_test` là phần đầu tiên của tệp âm thanh thử nghiệm mà chúng ta sẽ cung cấp cho máy tính làm input.

`y_test`,"nhãn" sẽ là phần còn lại của tệp âm thanh. Như vậy, máy tính sẽ sử dụng hồi quy tuyến tính để lấy ra phần bị thiếu của tệp âm thanh dựa trên dữ liệu huấn luyện mà nó nhận được!

Hãy lưu clip `test` ban đầu, clip mà bạn sắp xóa một nửa vào thư mục hiện tại để bạn có thể so sánh nó với clip 'đã vá' sau khi tạo. Bạn hẳn đã có `sample_rate` khi tải lên các tệp .wav: 

**sample rate là tần số lấy mẫu, là số giá trị để biểu thị âm thanh trong 1s, tần số mẫu càng cao thì âm thanh càng thi tiết**

In [18]:
wavfile.write('Original Test Clip.wav', sample_rate, test)

Chuẩn bị dữ liệu TEST bằng cách tạo lát `X_test`. Nó phải có các thuộc tính mẫu âm thanh `Provided_Portion` * `n_audio_samples`, được lấy từ tệp âm thanh thử nghiệm, hiện được lưu trữ trong biến `test`. Nói cách khác, lấy các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` ĐẦU TIÊN từ `test` và lưu trữ nó trong` X_test`. Điều này sẽ được thực hiện bằng cách sử dụng lập chỉ mục:

In [19]:
X_test = test[:int((1-Provided_Portion)*test.shape[0])]
Y_test = test[int((1-Provided_Portion)*test.shape[0]):]

X_test.shape, Y_test.shape

((3065,), (1022,))

Nếu các thuộc tính `Provided_Portion` * `n_audio_samples` đầu tiên được lưu trữ trong `X_test`, thì chúng ta cũng cần lấy các thuộc tính âm thanh _còn lại_ và lưu trữ chúng trong` y_test`. Với các thuộc tính còn lại được lưu trữ trong đó, chúng ta sẽ có thể R ^ 2 "chấm điểm" thuật toán đã làm tốt như thế nào trong việc hoàn thành tệp âm thanh.

cắt ra 1 phần để làm train_x 1 phần làm train_y

tỉ lệ 0.8 - 0.2

hiện có 1 đoạn âm thành dài 6273 feature

tách ra 1 phần để train 1 phần để làm label

Lặp lại quy trình tương tự cho `X_train`, `y_train`. Sự khác biệt duy nhất là:

1. Bạn sẽ nhận được dữ liệu âm thanh từ `train` thay vì từ `test`
2. Bạn có nhớ shape của `train` mà bạn đã in ra trước đó không? Bạn muốn thực hiện việc cắt này nhưng đối với TẤT CẢ các mẫu (quan sát). Đối với mỗ quan sát, bạn muốn chia các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` đầu tiên thành `X_train`, và phần còn lại chuyển vào `y_train`. Có thể thực hiện tất cả những điều này bằng lập chỉ mục thông thường ở 2 dòng code:

In [20]:
X_test.shape, Y_test.shape

((3065,), (1022,))

In [21]:
X_train = train[:, :int((1-Provided_Portion)*train.shape[1])]
Y_train = train[:, int((1-Provided_Portion)*train.shape[1]):]

X_train.shape, Y_train.shape

((49, 3065), (49, 1022))

SciKit-Learn sẽ 'tức giận' nếu bạn không cung cấp dữ liệu huấn luyện dưới dạng khung dữ liệu 2D có shape `[n_samples, n_features]`.

Vì vậy, nếu bạn chỉ có một MẪU, chẳng hạn như trường hợp của chúng ta với `X_test` và `y_test` thì bằng cách gọi `.reshape(1, -1)`, bạn có thể biến `[n_features]` thành `[1, n_features] `để đáp ứng SciKit-Learn.

Mặt khác, nếu bạn chỉ có một THUỘC TÍNH, bạn có thể gọi `.reshape (-1, 1)` trên dữ liệu của mình để biến `[n_samples]` thành `[n_samples, 1]`.

Định hình lại X_test và y_test thành [1, n_features]:

In [22]:
X_test = X_test.reshape(1,-1)
Y_test = Y_test.reshape(1,-1)

In [23]:
X_test.shape

(1, 3065)

In [24]:
Y_test.shape

(1, 1022)

In [25]:
Y_test

array([[4253, 5050, 4948, ...,  638,  463,  332]], dtype=int16)

In [26]:
X_train.shape

(49, 3065)

In [27]:
Y_train.shape

(49, 1022)

# Bắt đầu lại ở đây

Điều chỉnh mô hình của bạn bằng cách sử dụng dữ liệu huấn luyện và nhãn:

In [28]:
from sklearn.linear_model import LinearRegression
# Hồi quy này là một biến mới và đồng thời sẽ trở thành đối tượng của tuyến tính.
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

## 3. Đánh giá mô hình

Sử dụng mô hình để dự đoán `label` của `X_test`. Lưu trữ dự đoán kết quả trong biến `y_test_prediction`:

In [29]:
y_test_prediction = regressor.predict(X_test)

SciKit-Learn sẽ sử dụng float64 để tạo các dự đoán của bạn, vì vậy hãy đưa các giá trị đó trở lại int16, đó là những gì tệp .wav cần:

In [30]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

In [31]:
print ('Coefficients: ', regressor.coef_)
print ('Intercept: ',regressor.intercept_)

Coefficients:  [[ 4.67108708e-04  5.23655853e-05 -4.21577064e-04 ...  5.37261715e-03
   1.22999209e-02  1.80275869e-02]
 [ 3.31707834e-04  8.96135981e-05 -2.50559717e-04 ...  4.27282090e-05
   8.36604800e-03  1.60064465e-02]
 [ 1.45121989e-03  4.95827047e-04 -4.53225174e-04 ... -5.33747524e-03
   3.43733114e-03  1.23009576e-02]
 ...
 [ 2.13283915e-04  1.77043922e-04  1.46367342e-04 ...  2.22833034e-03
   1.45344777e-03  6.80209792e-04]
 [-2.19978194e-04  3.46974686e-05  2.55214941e-04 ...  2.83574465e-03
   1.50611165e-03  5.14780526e-05]
 [-3.53650197e-04 -1.91173453e-05  2.67932804e-04 ...  3.48322720e-03
   1.85501506e-03  7.96970187e-05]]
Intercept:  [ -692.5484125   -872.27308731 -1044.51179778 ...   192.18220607
    54.71038803  -159.65422944]


Chấm điểm dự đoán của bạn sẽ hoạt động tốt như thế nào bằng cách truyền dữ liệu và nhãn thử nghiệm `y_test`:

In [32]:
y_test_prediction

array([[-2318, -1819, -1367, ...,  -997, -1733, -2233]], dtype=int16)

In [33]:
Y_test

array([[4253, 5050, 4948, ...,  638,  463,  332]], dtype=int16)

In [34]:
from sklearn.metrics import r2_score

print("Mean absolute error: %.2f" % np.mean(np.absolute(y_test_prediction - Y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_test_prediction - Y_test) ** 2))
print("R2-score: %.2f" % r2_score(Y_test[0], y_test_prediction[0]) )

Mean absolute error: 1392.25
Residual sum of squares (MSE): 1314.55
R2-score: 1.00


Điểm R ^ 2 ngoại suy

In [35]:
# print("Extrapolation R^2 Score: ", score)

Hãy lấy phần `Provided_Portion` đầu tiên của clip thử nghiệm, phần mà bạn đã đưa vào mô hình hồi quy tuyến tính. Sau đó, kết hợp điều đó với thứ mà mô hình dự đoán đã tạo cho bạn và sau đó lưu đoạn âm thanh đã hoàn thành:

In [36]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])